In [1]:
import re
import json
import ast
import sys
import random
import json
import math
import os
import os.path
import sys
import csv

import numpy as np
from numpy import random as rd
import random 
from collections import Counter
import unicodedata
import string
import re
import traceback
# from joblib import Parallel, delayed
# import sklearn

from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.pipeline import Pipeline

from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import SGDClassifier
# from xgboost.sklearn import XGBClassifier
# import xgboost as xgb


from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

import pandas as pd

from random import shuffle

In [2]:

def get_text(tweet):
    tweetText = re.sub(r"(?:\@|https?\://)\S+", "", tweet) # r'(?:@[\w_]+)'
    tweetText = tweetText.strip().replace('rt', '').replace('\n', ' ').replace('\r', ' ').replace('\t', ' ')
    
    return tweetText
 
from numpy import random as rd

def get_statce_stats(emb, label):
    y_counts = {}
    y_Xs = {}
    y_indices = {}
    for i, (emb, label) in enumerate(zip(emb, label)):
        if label in y_counts:
            y_counts[label] = y_counts[label]  + 1
        else:
            y_counts[label] = 1    

        if label in y_Xs:
            y_Xs[label].append(emb)
        else:
            y_Xs[label] = [emb]  
            
        if label in y_indices:
            y_indices[label].append(i)
        else:
            y_indices[label] = [i]  

    print('lables : ', y_counts)
    
    return y_counts, y_Xs, y_indices

def balance_stance_classes(emb_vec, labels):
    y_counts, y_Xs, y_indices = get_statce_stats(emb_vec, labels)  
    
        
    majority_label = 0
    minority_label = 1
    majority_count = 0
    for key, val in y_counts.items():
        if val > majority_count:
            
            minority_label = majority_label
            majority_label = key
            majority_count= val
        else:
            minority_label = key
        
    new_emb_vec = []    
    new_labels = []
    for label in y_counts.keys():
        if label != majority_label:
            minority_label = label
            
            if y_counts[majority_label] > y_counts[minority_label]:
                additional_samples_count = y_counts[majority_label] - y_counts[minority_label]

                indices = rd.choice(list(y_indices[minority_label]), size=additional_samples_count, replace=True)
                for index in indices:
                    new_emb_vec.append(emb_vec[index])
                    new_labels.append(labels[index])
            

    y_counts, y_Xs, y_indices  = get_statce_stats(new_emb_vec + emb_vec, new_labels + labels)        
    

    return  new_emb_vec + emb_vec, new_labels + labels


In [3]:
# stance_to_num = {}
# # update data for labels to match with pheme
# # pheme label{'query': 0, 'comment': 1, 'deny': 3, 'support': 2}
# stance_to_num['Support'] = 2
# stance_to_num['Denial'] = 3
# stance_to_num['Comment'] = 1
# stance_to_num['Queries'] = 0

# print(stance_to_num)

In [4]:

from sklearn.model_selection import train_test_split

def get_text(tweet):
    tweetText = re.sub(r"(?:\@|https?\://)\S+", "", tweet) # r'(?:@[\w_]+)'
    tweetText = tweetText.strip().replace('rt', '').replace('#', '').replace('\n', ' ').replace('\r', ' ').replace('\t', ' ').replace('.', ' ').replace(',', ' ').replace('!', ' ').replace(':', ' ').replace('?', ' ?').lower()
    return tweetText

def add_to_dictionary(dictionary, key, count = 1):
    if key not in dictionary:
        dictionary[key] = count
    else:
        dictionary[key] += count
        
def add_key_value_to_dictionary(dictionary, key, value):
    if key not in dictionary:
        dictionary[key] = [value]
    else:
        dictionary[key].extend(value)

        
        
stance_to_num = {}
# update data for labels to match with pheme
# pheme label{'query': 0, 'comment': 1, 'deny': 3, 'support': 2}
stance_to_num['Support'] = 0
stance_to_num['Denial'] = 1
stance_to_num['Comment'] = 2
stance_to_num['Queries'] = 3

print(stance_to_num)


event_to_num  = {}
event_useful_tweet_ids = {}
useful_tweet_ids = {}
stance_count = {}
label_reply_ids = {}


df = pd.read_csv('./data/Contentious_pairs.csv' ,  encoding="utf-8", 
                 dtype={"response_id":str, "target_id":str} ).fillna('')
    
    

    
df = df[df['event']!= 'General_Terms']    

df_event_train, df_event_test = train_test_split(df, test_size=0.3)

labeled_tweets_of_intereset = {}
    
    
    
def get_classification_data(df, class_count = 2):    
    
    classification_data = []
    data_labels = []

    
    for response_id, target_id, response_text, target_text, stance, event, interaction_type in zip(df['response_id'], df['target_id'], 
                                                                  df['response_text'], df['target_text']
                                                                 , df['label'], df['event'], df['interaction_type']):

        if (class_count==2 and stance !='Comment') or class_count==3:

            if len(response_text)> 1 and len(target_text) > 1 and stance != 'Missing'  and stance !='Queries' : # and  stance !='Queries' and  stance !='Comment'


                if response_id not in labeled_tweets_of_intereset:
                    labeled_tweets_of_intereset[response_id] = 1

                if target_id not in labeled_tweets_of_intereset:
                    labeled_tweets_of_intereset[target_id] = 1

                if stance not in stance_to_num:
                    stance_to_num[stance]  = len(stance_to_num)

                if event not in event_to_num:
                    event_to_num[event] = len(event_to_num)

                if event not in event_useful_tweet_ids:
                    event_useful_tweet_ids[event] = {}

                if response_id not in useful_tweet_ids:
                    useful_tweet_ids[response_id] = 1
                else:
                    useful_tweet_ids[response_id] += 1

                if target_id not in useful_tweet_ids:
                    useful_tweet_ids[target_id] = 1
                else:
                    useful_tweet_ids[target_id] += 1

                add_to_dictionary(stance_count, stance_to_num[stance])

                classification_data.append((target_id, response_id, stance_to_num[stance], event_to_num[event], interaction_type, get_text(response_text), get_text(target_text)))
                data_labels.append(stance_to_num[stance])
#                 add_key_value_to_dictionary(label_reply_ids, str(response_id), len(data_labels))

    print(len(classification_data))   
    
    return classification_data, data_labels


classification_data, data_labels = get_classification_data(df_event_train, class_count = 2)
classification_data_test, data_labels_test = get_classification_data(df_event_test, class_count = 2)
print('2 class data ', set(data_labels_test), len(classification_data), len(data_labels), len(classification_data_test), len(data_labels_test))


classification_data_3_class, data_labels_3_class = get_classification_data(df_event_train, class_count = 3)
classification_data_test_3_class, data_labels_test_3_class = get_classification_data(df_event_test, class_count = 3)
print('3 class data ', set(data_labels_test_3_class), len(classification_data_3_class), len(data_labels_3_class), len(classification_data_test_3_class), len(data_labels_test_3_class))


print('stance_to_num: ', stance_to_num)
print('event_to_num: ', event_to_num)
print('labeled_tweets_of_intereset: ' , len(labeled_tweets_of_intereset))   


labeled_tweets_of_interest_to_user_id = {}
with open ('./data/tweetid_to_users_v2.json', 'r') as f_read:
    tweetid_to_users = json.load(f_read)

    for tweetid, userid in tweetid_to_users.items():
        if str(tweetid) in labeled_tweets_of_intereset:
            labeled_tweets_of_interest_to_user_id[str(tweetid)] = str(userid)

print('labeled_tweets_of_interest_to_user_id: ', len(labeled_tweets_of_interest_to_user_id))  



{'Support': 0, 'Denial': 1, 'Comment': 2, 'Queries': 3}
1705
721
2 class data  {0, 1} 1705 1705 721 721
2187
931
3 class data  {0, 1, 2} 2187 2187 931 931
stance_to_num:  {'Support': 0, 'Denial': 1, 'Comment': 2, 'Queries': 3}
event_to_num:  {'Santa_Fe_Shooting': 0, 'Student_Marches': 1, 'Iran_Deal': 2}
labeled_tweets_of_intereset:  5449
labeled_tweets_of_interest_to_user_id:  5261


## Get event data, split it in train (70%) and test (30%)

In [5]:
def get_all_events_data(interaction_type_to_print = "Reply", class_count = 2):

    if class_count == 2:
        
        shuffle(classification_data)
        balanced_classification_data, balanced_labels = balance_stance_classes(classification_data, data_labels)

        
        test_data = classification_data_test
        train_data = balanced_classification_data
        print('2 class ', len(train_data), len(test_data))        
        
        
    else:
        shuffle(classification_data_3_class)
        balanced_classification_data_3_class, balanced_labels_3_class = balance_stance_classes(classification_data_test_3_class, data_labels_test_3_class)
        
        test_data = classification_data_test_3_class
        train_data = balanced_classification_data_3_class
        
        print('3 class ', len(train_data), len(test_data))                
        

     #"Quote" # Reply

    all_events_data = {}    

    for event_name, num in event_to_num.items():
        event_data = {}

        event_data['ids'] = []
        event_data['text'] = []
        event_data['train_ind'] = []
        event_data['val_ind'] = []
        event_data['test_ind'] = []

        event_data['info']  = []

        i = 0 
        for data_item in test_data:
            source_id, reply_id, stance_num, event_num, interaction_type, response_text, target_text = data_item

            if interaction_type_to_print == interaction_type or interaction_type_to_print =="ALL":

                if num == event_num: # validation event
                    
#                     if not filter_comment_and_query or (filter_comment_and_query and stance_num> 1): #filter
                    event_data['test_ind'].append(i)
                    i += 1

                    event_data['ids'].append({'src': source_id, 'reply': reply_id})
                    event_data['text'].append({'src': target_text, 'reply': response_text})
    #                 label_vec = [0]*len(stance_to_num)
    #                 label_vec[stance_num] = stance_to_num

                    event_data['info'].append({'label':stance_num})
                    

        for data_item in train_data:
            source_id, reply_id, stance_num, event_num, interaction_type, response_text, target_text = data_item

            if interaction_type_to_print == interaction_type or interaction_type_to_print =="ALL":

                if num == event_num: # not validation event
#                     if not filter_comment_and_query or (filter_comment_and_query and stance_num> 1): #filter                    
                    event_data['train_ind'].append(i )            
                    i += 1

                    event_data['ids'].append({'src': source_id, 'reply': reply_id})
                    event_data['text'].append({'src': target_text, 'reply': response_text})
    #                 label_vec = [0]*len(stance_to_num)
    #                 label_vec[stance_num] = stance_to_num

                    event_data['info'].append({'label':stance_num})

        all_events_data[event_name] = event_data
        
    return all_events_data
    




In [6]:
all_events_data_all = get_all_events_data(interaction_type_to_print = "ALL", class_count = 2)
print(len(all_events_data_all['Student_Marches']))  


all_events_data_all_3_class = get_all_events_data(interaction_type_to_print = "ALL",  class_count = 3)
print(len(all_events_data_all_3_class['Student_Marches']), 
      len(all_events_data_all_3_class['Santa_Fe_Shooting']),
      len(all_events_data_all_3_class['Iran_Deal']))  


lables :  {0: 754, 1: 951}
lables :  {0: 951, 1: 951}
2 class  1902 721
6
lables :  {1: 404, 2: 210, 0: 317}
lables :  {2: 404, 0: 404, 1: 404}
3 class  1212 931
6 6 6


## Define LSTM/RNN model

In [23]:
from __future__ import print_function
from functools import reduce
import json
import os
import re
import tarfile
import tempfile

import numpy as np
np.random.seed(1337)  # for reproducibility

import keras
import keras.backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import   Dense, Input, Dropout, TimeDistributed, concatenate

# from keras.layers.normalization import BatchNormalization
# from keras.layers.wrappers import Bidirectional
from keras.models import Model
# from keras.preprocessing.sequence import pad_sequences
# from keras.preprocessing.text import Tokenizer
from keras.regularizers import l2
from keras.utils import np_utils
from keras.layers import LSTM
from keras.layers import GRU
from keras.layers import SimpleRNN
from keras.layers import ConvLSTM1D
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
# from keras.layers.embeddings import Embedding
from keras.layers import Embedding
from tensorflow.keras.layers import BatchNormalization


from sklearn.metrics import f1_score
from keras import backend as K
import os
    
    

In [30]:
def extract_tokens_from_binary_parse(parse):
    return parse.replace('(', ' ').replace(')', ' ').replace('-LRB-', '(').replace('-RRB-', ')').split()

def yield_examples(fn, skip_no_majority=True, limit=None):
  for i, line in enumerate(open(fn)):
    if limit and i > limit:
      break
    data = json.loads(line)
    label = data['gold_label']
    s1 = ' '.join(extract_tokens_from_binary_parse(data['sentence1_binary_parse']))
    s2 = ' '.join(extract_tokens_from_binary_parse(data['sentence2_binary_parse']))
    if skip_no_majority and label == '-':
      continue
    yield (label, s1, s2)

    
def yield_new_examples(replies, sources, labels, limit=None):
    all_data = (labels, sources, replies)
    for i, data in enumerate(zip(labels, sources, replies)):
        if limit and i > limit:
            break

            
        yield (data[0], data[1], data[2])


def get_data(replies, sources, labels, limit=None, shuffle_data=True, LABEL_COUNT = 2):
    
    raw_data = list(yield_new_examples(replies, sources, labels, limit=limit))
    
    if shuffle_data:
        shuffle(raw_data)

    left = [s1 for _, s1, s2 in raw_data]
    right = [s2 for _, s1, s2 in raw_data]
    print(max(len(x.split()) for x in left))
    print(max(len(x.split()) for x in right))

    #   LABELS = {'contradiction': 0, 'neutral': 1, 'entailment': 2}
    Y = np.array([l   for l, s1, s2 in raw_data])
    Y = np_utils.to_categorical(Y, LABEL_COUNT)

    return left, right, Y


def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


def get_accuracy_estimate(truths, predicted):
#     print(type(truths))

    print(classification_report(truths, predicted))
#     print('Classifier Accuracy: ', accuracy_score(truths, predicted))

    print('f1_score micro: ', f1_score(truths, predicted, average='micro'))
    print('f1_score macro: ', f1_score(truths, predicted, average='macro'))    
    print('f1_score wieghted: ', f1_score(truths, predicted, average='weighted'))        

    return f1_score(truths, predicted, average='micro'), f1_score(truths, predicted, average='macro'), f1_score(truths, predicted, average='weighted')


def train_and_test(train_sources, train_replies, train_labels,
                   test_sources, test_replies, test_labels,
                  LABELS = {0: 0, 1:1},
                  MODEL = 'LSTM'):


    all_test = get_data(test_replies, 
                    test_sources,
                    test_labels, 
                    shuffle_data = False,
                    LABEL_COUNT = len(LABELS))    
    
#     training = get_data(train_replies, 
#                         train_sources,
#                         train_labels)


    validation_count = int(0.1 * len(train_replies))     
    validation = get_data(train_replies[0:validation_count]
                          ,train_sources[0:validation_count]
                          ,train_labels[0:validation_count],
                         LABEL_COUNT  = len(LABELS))

    training = get_data(train_replies[validation_count+1:], 
                    train_sources[validation_count+1:],
                    train_labels[validation_count+1:],
                     LABEL_COUNT=    len(LABELS))

    
    test = get_data(test_replies, 
                    test_sources,
                    test_labels,
                    LABEL_COUNT = len(LABELS))


    tokenizer = Tokenizer(lower=False, filters='')
    tokenizer.fit_on_texts(training[0] + training[1])

    # Lowest index from the tokenizer is 1 - we need to include 0 in our vocab count
    VOCAB = len(tokenizer.word_counts) + 1
     # , {0: 0, 1: 1, 2: 2, 3:3} {'Comment': 1, 'Queries': 0, 'Support': 2, 'Denial': 3}

#     RNN = None 
    if MODEL == 'LSTM':
        RNN = LSTM
    elif MODEL == 'GRU':
        RNN = GRU
    elif MODEL == 'ConvLSTM1D':
        RNN = ConvLSTM1D(filters=3, kernel_size= 5)
    else:
        RNN = SimpleRNN 
        
        
#     RNN = lambda *args, **kwargs: Bidirectional(recurrent.LSTM(*args, **kwargs))
#     RNN = recurrent.GRU
#     RNN = lambda *args, **kwargs: Bidirectional(recurrent.GRU(*args, **kwargs))


#     LAYERS = 1 #2
    USE_GLOVE = True
    TRAIN_EMBED = False
    EMBED_HIDDEN_SIZE = 200
    SENT_HIDDEN_SIZE = 200
    BATCH_SIZE = 64
    PATIENCE = 4 # 8
    MAX_EPOCHS = 20 #100
    MAX_LEN = 42
    DP = 0.3
#     L2 = 1e-5 #4e-6
    ACTIVATION = 'relu'
    OPTIMIZER = 'rmsprop'
    print('RNN / Embed / Sent = {}, {}, {}'.format(RNN, EMBED_HIDDEN_SIZE, SENT_HIDDEN_SIZE))
    print('GloVe / Trainable Word Embeddings = {}, {}'.format(USE_GLOVE, TRAIN_EMBED))

    to_seq = lambda X: pad_sequences(tokenizer.texts_to_sequences(X), maxlen=MAX_LEN)
    prepare_data = lambda data: (to_seq(data[0]), to_seq(data[1]), data[2])

    training = prepare_data(training)
    validation = prepare_data(validation)
    test = prepare_data(test)
    all_test = prepare_data(all_test)

    print('Build model...')
    print('Vocab size =', VOCAB)

    GLOVE_STORE = './precomputed_glove1.weights'
    if USE_GLOVE:
      if True : # not os.path.exists(GLOVE_STORE + '.npy')
        print('Computing GloVe')

        embeddings_index = {}
        f = open('./data/glove.twitter.27B.200d.txt') # glove.840B.300d.txt') #glove.twitter.27B.100d.txt') #+ 'glove.840B.300d.txt')
        for line in f:
            try:
                values = line.split(' ')
                word = values[0]
                coefs = np.asarray(values[1:], dtype='float32')
                embeddings_index[word] = coefs
            except:
                print('Error: ', values[0], values[1:])
        f.close()

        # prepare embedding matrix
        embedding_matrix = np.zeros((VOCAB, EMBED_HIDDEN_SIZE))
        for word, i in tokenizer.word_index.items():
          embedding_vector = embeddings_index.get(word)
          if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
#           else:
#             print('Missing from GloVe: {}'.format(word))

        np.save(GLOVE_STORE, embedding_matrix)

      print('Loading GloVe')
      embedding_matrix = np.load(GLOVE_STORE + '.npy')

      print('Total number of null word embeddings:')
      print(np.sum(np.sum(embedding_matrix, axis=1) == 0))

      embed = Embedding(VOCAB, EMBED_HIDDEN_SIZE, weights=[embedding_matrix], input_length=MAX_LEN, trainable=TRAIN_EMBED)
    else:
      embed = Embedding(VOCAB, EMBED_HIDDEN_SIZE, input_length=MAX_LEN)

#     rnn_kwargs = dict(output_dim=SENT_HIDDEN_SIZE, dropout_W=DP, dropout_U=DP)
    
    rnn_kwargs = dict(dropout=DP)    
    SumEmbeddings = keras.layers.core.Lambda(lambda x: K.sum(x, axis=1), output_shape=(SENT_HIDDEN_SIZE, ))

    translate = TimeDistributed(Dense(SENT_HIDDEN_SIZE, activation=ACTIVATION))

    premise = Input(shape=(MAX_LEN,), dtype='int32')
    hypothesis = Input(shape=(MAX_LEN,), dtype='int32')

    prem = embed(premise)
    hypo = embed(hypothesis)

    prem = translate(prem)
    hypo = translate(hypo)
        
    rnn = SumEmbeddings if not RNN else RNN(SENT_HIDDEN_SIZE, return_sequences=False, **rnn_kwargs)
    
    prem = rnn(prem)
    hypo = rnn(hypo)
    prem = BatchNormalization()(prem)
    hypo = BatchNormalization()(hypo)

    joint = concatenate([prem, hypo]) #, mode='concat'
    joint = Dropout(DP)(joint)

    for i in range(LAYERS):
      joint = Dense(2 * SENT_HIDDEN_SIZE, activation=ACTIVATION, kernel_regularizer=l2(L2) if L2 else None)(joint)
      joint = Dropout(DP)(joint)
      joint = BatchNormalization()(joint)

    pred = Dense(len(LABELS), activation='softmax')(joint)

    model = Model([premise, hypothesis], pred)
    
    model.compile(optimizer=OPTIMIZER, 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy', 
                           f1_m])

#     model.summary()

    print('Training')
    _, tmpfn = tempfile.mkstemp()
    # Save the best model during validation and bail out of training early if we're not improving
    callbacks = [EarlyStopping(patience = PATIENCE), ModelCheckpoint(tmpfn, save_best_only=True, save_weights_only=True)]
    model.fit([training[0], training[1]], training[2], batch_size=BATCH_SIZE, epochs= MAX_EPOCHS, validation_data=([validation[0], validation[1]], validation[2]), callbacks=callbacks)

    # Restore the best found model during validation
    model.load_weights(tmpfn)

    loss, acc, f1 = model.evaluate([test[0], test[1]], test[2], batch_size=BATCH_SIZE)
    print('Test loss / test accuracy / test f1 = {:.4f} / {:.4f} / {:.4f}'.format(loss, acc, f1))    
    
    
    y_proba = model.predict([all_test[0], all_test[1]], verbose=1)
#     x_vec = np_utils.to_categorical(y_proba, 4)
    y_classes = y_proba.argmax(axis= -1)
    
#     print(classification_report(, y_classes))    
    y_truths = all_test[2].argmax(axis= -1)
    f1_micro, f1_macro,  f1_weighted = get_accuracy_estimate(y_truths, y_classes)
#     print('f1_val micro', f1_val)
    
    keras.backend.clear_session()
    os.remove(tmpfn)
    
    return f1_micro, f1_macro,  f1_weighted, y_truths, y_classes    

import json


def write_result_to_file(result_string, path):
    with open(path, 'a') as f_write:
        f_write.write("{}\n".format(result_string))


## 2 Classs LSTM Classification

In [32]:


        
nn_event_score = {}

for LAYERS in [1]: # , 2
    for L2 in [1e-5]:  # , 1e-4, 1e-6
        results = {'LAYERS': str(LAYERS), 'L2': L2}
        for i in range(10):
            for data_name, all_events_data in [
                        #                 ('reply', all_events_data_reply),
                                            ('all', all_events_data_all),
#                                                ('quote', all_events_data_quote)
                                              ]:
#                                                ('reply', all_events_data_reply)]:
                one_type_result = {}
                for event_name, all_event_data in all_events_data.items():
                    if True: #event_name == 'Student_Marches': #True #Student_Marches
                        print('i ', str(i), 'Event_name', event_name, 'LAYERS: ', str(LAYERS), 'L2: ', L2, len(all_event_data['train_ind']), len(all_event_data['test_ind']))

                        train_replies = []
                        train_sources = []
                        train_labels = []
                        for train_index in all_event_data['train_ind']:
                            train_replies.append(all_event_data['text'][train_index]['reply'])
                            train_sources.append(all_event_data['text'][train_index]['src'])
                            train_labels.append(all_event_data['info'][train_index]['label'])

                        test_replies = []
                        test_sources = []
                        test_labels = []
                        for test_index in all_event_data['test_ind']:
                            test_replies.append(all_event_data['text'][test_index]['reply'])
                            test_sources.append(all_event_data['text'][test_index]['src'])
                            test_labels.append(all_event_data['info'][test_index]['label'])


                        f1_micro, f1_macro,  f1_weighted, y_truths, y_classes = train_and_test( train_replies, train_sources, train_labels,
                                   test_replies, test_sources, test_labels, LABELS = {0: 0, 1:1})

        #                 f1_macro = train_and_test(train_sources, train_replies, train_labels,
        #                            test_sources, test_replies, test_labels)


                        one_type_result[event_name] = {'f1_micro': f1_micro,
                                                       'f1_macro': f1_macro,
                                                       'f1_weighted': f1_weighted,
                                                       'y_truths': ' '.join([str(i) for i in y_truths]), 
                                                       'y_classes': ' '.join([str(i) for i in y_classes])}

                    if event_name not in nn_event_score:
                        nn_event_score[event_name] = [f1_macro]
                    else:
                        nn_event_score[event_name].append(f1_macro)


                    results[data_name + '_' + str(i)] = one_type_result  

        results_str = json.dumps(results)
        write_result_to_file(results_str, './results/Replies_stance_classifier_using_LSTM_RNN_2_class.csv')


"{:.2f}&{:.2f}&{:.2f}&{:.2f}".format(np.mean(nn_event_score['Student_Marches']),
                                     np.mean(nn_event_score['Santa_Fe_Shooting']),
                                     np.mean(nn_event_score['Iran_Deal']),
                             np.mean([nn_event_score['Student_Marches'], nn_event_score['Santa_Fe_Shooting'], nn_event_score['Iran_Deal']]))



i  0 Event_name Santa_Fe_Shooting LAYERS:  1 L2:  1e-05 761 273
57
54
61
53
61
63
57
54
RNN / Embed / Sent = <class 'keras.layers.rnn.lstm.LSTM'>, 200, 200
GloVe / Trainable Word Embeddings = True, False
Build model...
Vocab size = 5389
Computing GloVe
Loading GloVe
Total number of null word embeddings:
1218
Training
Epoch 1/20
11/11 [==============================] - 5s 177ms/step - loss: 1.0210 - accuracy: 0.5424 - f1_m: 0.5425 - val_loss: 0.6815 - val_accuracy: 0.5395 - val_f1_m: 0.5234
Epoch 2/20
11/11 [==============================] - 1s 116ms/step - loss: 0.7225 - accuracy: 0.6740 - f1_m: 0.6761 - val_loss: 0.6579 - val_accuracy: 0.6579 - val_f1_m: 0.5937
Epoch 3/20
11/11 [==============================] - 1s 117ms/step - loss: 0.6578 - accuracy: 0.6813 - f1_m: 0.6800 - val_loss: 0.6444 - val_accuracy: 0.6316 - val_f1_m: 0.6120
Epoch 4/20
11/11 [==============================] - 1s 114ms/step - loss: 0.5426 - accuracy: 0.7500 - f1_m: 0.7487 - val_loss: 0.6186 - val_accuracy: 0.8

'0.61&0.56&0.61&0.59'

## 3 Class LSTM classification

In [33]:

        
        

nn_event_score2 = {}

for LAYERS in [1]: # , 2
    for L2 in [1e-5]:  # , 1e-4, 1e-6
        results = {'LAYERS': str(LAYERS), 'L2': L2}
        for i in range(10):
            for data_name, all_events_data in [
                        #                 ('reply', all_events_data_reply),
                                            ('all', all_events_data_all_3_class),
#                                                ('quote', all_events_data_quote)
                                              ]:
#                                                ('reply', all_events_data_reply)]:
                one_type_result = {}
                for event_name, all_event_data in all_events_data.items():
                    if True: #event_name == 'Student_Marches': #True #Student_Marches
                        print('i ', str(i), 'Event_name', event_name, 'LAYERS: ', str(LAYERS), 'L2: ', L2, len(all_event_data['train_ind']), len(all_event_data['test_ind']))

                        train_replies = []
                        train_sources = []
                        train_labels = []
                        for train_index in all_event_data['train_ind']:
                            train_replies.append(all_event_data['text'][train_index]['reply'])
                            train_sources.append(all_event_data['text'][train_index]['src'])
                            train_labels.append(all_event_data['info'][train_index]['label'])

                        test_replies = []
                        test_sources = []
                        test_labels = []
                        for test_index in all_event_data['test_ind']:
                            test_replies.append(all_event_data['text'][test_index]['reply'])
                            test_sources.append(all_event_data['text'][test_index]['src'])
                            test_labels.append(all_event_data['info'][test_index]['label'])


                        f1_micro, f1_macro,  f1_weighted, y_truths, y_classes = train_and_test( train_replies, train_sources, train_labels,
                                   test_replies, test_sources, test_labels,LABELS = {0:0, 1:1, 2: 2})

        #                 f1_macro = train_and_test(train_sources, train_replies, train_labels,
        #                            test_sources, test_replies, test_labels)


                        one_type_result[event_name] = {'f1_micro': f1_micro,
                                                       'f1_macro': f1_macro,
                                                       'f1_weighted': f1_weighted,
                                                       'y_truths': ' '.join([str(i) for i in y_truths]), 
                                                       'y_classes': ' '.join([str(i) for i in y_classes])}

                    if event_name not in nn_event_score2:
                        nn_event_score2[event_name] = [f1_macro]
                    else:
                        nn_event_score2[event_name].append(f1_macro)


                    results[data_name + '_' + str(i)] = one_type_result  

        results_str = json.dumps(results)
        write_result_to_file(results_str, './results/Replies_stance_classifier_using_LSTM_RNN_3_class.csv')        
   



"{:.2f}&{:.2f}&{:.2f}&{:.2f}".format(np.mean(nn_event_score2['Student_Marches']),
                                     np.mean(nn_event_score2['Santa_Fe_Shooting']),
                                     np.mean(nn_event_score2['Iran_Deal']),
                             np.mean([nn_event_score2['Student_Marches'], nn_event_score2['Santa_Fe_Shooting'], nn_event_score2['Iran_Deal']]))



i  0 Event_name Santa_Fe_Shooting LAYERS:  1 L2:  1e-05 427 335
57
59
57
49
57
59
57
59
RNN / Embed / Sent = <class 'keras.layers.rnn.lstm.LSTM'>, 200, 200
GloVe / Trainable Word Embeddings = True, False
Build model...
Vocab size = 3632
Computing GloVe
Loading GloVe
Total number of null word embeddings:
676
Training
Epoch 1/20
6/6 [==============================] - 4s 233ms/step - loss: 1.5293 - accuracy: 0.3594 - f1_m: 0.3375 - val_loss: 1.5989 - val_accuracy: 0.0000e+00 - val_f1_m: 0.0000e+00
Epoch 2/20
6/6 [==============================] - 1s 109ms/step - loss: 1.0248 - accuracy: 0.5547 - f1_m: 0.5433 - val_loss: 1.4667 - val_accuracy: 0.0000e+00 - val_f1_m: 0.0000e+00
Epoch 3/20
6/6 [==============================] - 1s 100ms/step - loss: 0.8805 - accuracy: 0.6198 - f1_m: 0.6150 - val_loss: 1.7061 - val_accuracy: 0.0000e+00 - val_f1_m: 0.0000e+00
Epoch 4/20
6/6 [==============================] - 1s 107ms/step - loss: 0.7748 - accuracy: 0.6979 - f1_m: 0.6818 - val_loss: 1.6431 - va

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


i  0 Event_name Student_Marches LAYERS:  1 L2:  1e-05 283 210
55
53
50
47
55
53
55
53
RNN / Embed / Sent = <class 'keras.layers.rnn.lstm.LSTM'>, 200, 200
GloVe / Trainable Word Embeddings = True, False
Build model...
Vocab size = 2865
Computing GloVe
Loading GloVe
Total number of null word embeddings:
543
Training
Epoch 1/20
4/4 [==============================] - 5s 339ms/step - loss: 1.4698 - accuracy: 0.4055 - f1_m: 0.3827 - val_loss: 1.0801 - val_accuracy: 0.4286 - val_f1_m: 0.0000e+00
Epoch 2/20
4/4 [==============================] - 0s 116ms/step - loss: 0.9328 - accuracy: 0.5984 - f1_m: 0.5796 - val_loss: 1.4179 - val_accuracy: 0.0000e+00 - val_f1_m: 0.0000e+00
Epoch 3/20
4/4 [==============================] - 0s 125ms/step - loss: 0.7909 - accuracy: 0.6890 - f1_m: 0.6792 - val_loss: 1.2537 - val_accuracy: 0.0357 - val_f1_m: 0.0000e+00
Epoch 4/20
4/4 [==============================] - 0s 118ms/step - loss: 0.7075 - accuracy: 0.6772 - f1_m: 0.6686 - val_loss: 1.3747 - val_accuracy

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


i  1 Event_name Santa_Fe_Shooting LAYERS:  1 L2:  1e-05 427 335
57
59
57
49
57
59
57
59
RNN / Embed / Sent = <class 'keras.layers.rnn.lstm.LSTM'>, 200, 200
GloVe / Trainable Word Embeddings = True, False
Build model...
Vocab size = 3632
Computing GloVe
Loading GloVe
Total number of null word embeddings:
676
Training
Epoch 1/20
6/6 [==============================] - 4s 301ms/step - loss: 1.5909 - accuracy: 0.3411 - f1_m: 0.3273 - val_loss: 1.2039 - val_accuracy: 0.0238 - val_f1_m: 0.0000e+00
Epoch 2/20
6/6 [==============================] - 1s 106ms/step - loss: 1.1149 - accuracy: 0.5312 - f1_m: 0.5143 - val_loss: 1.4423 - val_accuracy: 0.0000e+00 - val_f1_m: 0.0000e+00
Epoch 3/20
6/6 [==============================] - 1s 105ms/step - loss: 0.9050 - accuracy: 0.6276 - f1_m: 0.5983 - val_loss: 1.4764 - val_accuracy: 0.0000e+00 - val_f1_m: 0.0000e+00
Epoch 4/20
6/6 [==============================] - 1s 106ms/step - loss: 0.6827 - accuracy: 0.7083 - f1_m: 0.6975 - val_loss: 1.3481 - val_ac

...

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


57
59
57
49
57
59
57
59
RNN / Embed / Sent = <class 'keras.layers.rnn.lstm.LSTM'>, 200, 200
GloVe / Trainable Word Embeddings = True, False
Build model...
Vocab size = 3632
Computing GloVe
Loading GloVe
Total number of null word embeddings:
676
Training
Epoch 1/20
6/6 [==============================] - 4s 241ms/step - loss: 1.5353 - accuracy: 0.3776 - f1_m: 0.3476 - val_loss: 1.5676 - val_accuracy: 0.0000e+00 - val_f1_m: 0.0000e+00
Epoch 2/20
6/6 [==============================] - 1s 114ms/step - loss: 1.0237 - accuracy: 0.5312 - f1_m: 0.5249 - val_loss: 1.4137 - val_accuracy: 0.0000e+00 - val_f1_m: 0.0000e+00
Epoch 3/20
6/6 [==============================] - 1s 106ms/step - loss: 0.8718 - accuracy: 0.6172 - f1_m: 0.6002 - val_loss: 1.7254 - val_accuracy: 0.0000e+00 - val_f1_m: 0.0000e+00
Epoch 4/20
6/6 [==============================] - 1s 107ms/step - loss: 0.6585 - accuracy: 0.7161 - f1_m: 0.7096 - val_loss: 1.9052 - val_accuracy: 0.0000e+00 - val_f1_m: 0.0000e+00
Epoch 5/20
6/6 [==

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


i  9 Event_name Iran_Deal LAYERS:  1 L2:  1e-05 502 386
58
56
54
52
58
56
58
56
RNN / Embed / Sent = <class 'keras.layers.rnn.lstm.LSTM'>, 200, 200
GloVe / Trainable Word Embeddings = True, False
Build model...
Vocab size = 4247
Computing GloVe
Loading GloVe
Total number of null word embeddings:
890
Training
Epoch 1/20
8/8 [==============================] - 4s 190ms/step - loss: 1.6353 - accuracy: 0.3459 - f1_m: 0.2807 - val_loss: 1.0004 - val_accuracy: 0.2200 - val_f1_m: 0.0000e+00
Epoch 2/20
8/8 [==============================] - 1s 103ms/step - loss: 1.1274 - accuracy: 0.5144 - f1_m: 0.4852 - val_loss: 1.0653 - val_accuracy: 0.4600 - val_f1_m: 0.0000e+00
Epoch 3/20
8/8 [==============================] - 1s 105ms/step - loss: 0.9728 - accuracy: 0.5876 - f1_m: 0.5842 - val_loss: 1.0605 - val_accuracy: 0.0000e+00 - val_f1_m: 0.0000e+00
Epoch 4/20
8/8 [==============================] - 1s 102ms/step - loss: 0.9958 - accuracy: 0.5787 - f1_m: 0.5519 - val_loss: 1.0091 - val_accuracy: 0.68

'0.41&0.34&0.33&0.36'

## 2 Class GRU Classification

In [34]:


        
nn_event_score = {}

for LAYERS in [1]: # , 2
    for L2 in [1e-5]:  # , 1e-4, 1e-6
        results = {'LAYERS': str(LAYERS), 'L2': L2}
        for i in range(10):
            for data_name, all_events_data in [
                        #                 ('reply', all_events_data_reply),
                                            ('all', all_events_data_all),
#                                                ('quote', all_events_data_quote)
                                              ]:
#                                                ('reply', all_events_data_reply)]:
                one_type_result = {}
                for event_name, all_event_data in all_events_data.items():
                    if True: #event_name == 'Student_Marches': #True #Student_Marches
                        print('i ', str(i), 'Event_name', event_name, 'LAYERS: ', str(LAYERS), 'L2: ', L2, len(all_event_data['train_ind']), len(all_event_data['test_ind']))

                        train_replies = []
                        train_sources = []
                        train_labels = []
                        for train_index in all_event_data['train_ind']:
                            train_replies.append(all_event_data['text'][train_index]['reply'])
                            train_sources.append(all_event_data['text'][train_index]['src'])
                            train_labels.append(all_event_data['info'][train_index]['label'])

                        test_replies = []
                        test_sources = []
                        test_labels = []
                        for test_index in all_event_data['test_ind']:
                            test_replies.append(all_event_data['text'][test_index]['reply'])
                            test_sources.append(all_event_data['text'][test_index]['src'])
                            test_labels.append(all_event_data['info'][test_index]['label'])


                        f1_micro, f1_macro,  f1_weighted, y_truths, y_classes = train_and_test( train_replies, train_sources, train_labels,
                                   test_replies, test_sources, test_labels, LABELS = {0: 0, 1:1}, MODEL = 'GRU')

        #                 f1_macro = train_and_test(train_sources, train_replies, train_labels,
        #                            test_sources, test_replies, test_labels)


                        one_type_result[event_name] = {'f1_micro': f1_micro,
                                                       'f1_macro': f1_macro,
                                                       'f1_weighted': f1_weighted,
                                                       'y_truths': ' '.join([str(i) for i in y_truths]), 
                                                       'y_classes': ' '.join([str(i) for i in y_classes])}

                    if event_name not in nn_event_score:
                        nn_event_score[event_name] = [f1_macro]
                    else:
                        nn_event_score[event_name].append(f1_macro)


                    results[data_name + '_' + str(i)] = one_type_result  

        results_str = json.dumps(results)
        write_result_to_file(results_str, './results/Replies_stance_classifier_using_GRU_2_class.csv')


"{:.2f}&{:.2f}&{:.2f}&{:.2f}".format(np.mean(nn_event_score['Student_Marches']),
                                     np.mean(nn_event_score['Santa_Fe_Shooting']),
                                     np.mean(nn_event_score['Iran_Deal']),
                             np.mean([nn_event_score['Student_Marches'], nn_event_score['Santa_Fe_Shooting'], nn_event_score['Iran_Deal']]))



i  0 Event_name Santa_Fe_Shooting LAYERS:  1 L2:  1e-05 761 273
57
54
61
53
61
63
57
54
RNN / Embed / Sent = <class 'keras.layers.rnn.gru.GRU'>, 200, 200
GloVe / Trainable Word Embeddings = True, False
Build model...
Vocab size = 5389
Computing GloVe
Loading GloVe
Total number of null word embeddings:
1218
Training
Epoch 1/20
11/11 [==============================] - 5s 157ms/step - loss: 1.0604 - accuracy: 0.5278 - f1_m: 0.5276 - val_loss: 0.6652 - val_accuracy: 0.5395 - val_f1_m: 0.4896
Epoch 2/20
11/11 [==============================] - 1s 100ms/step - loss: 0.7410 - accuracy: 0.6243 - f1_m: 0.6265 - val_loss: 0.7252 - val_accuracy: 0.5395 - val_f1_m: 0.4896
Epoch 3/20
11/11 [==============================] - 1s 102ms/step - loss: 0.6442 - accuracy: 0.6988 - f1_m: 0.6977 - val_loss: 0.6739 - val_accuracy: 0.5395 - val_f1_m: 0.4896
Epoch 4/20
11/11 [==============================] - 1s 105ms/step - loss: 0.5715 - accuracy: 0.7485 - f1_m: 0.7466 - val_loss: 0.6345 - val_accuracy: 0.552

'0.61&0.57&0.61&0.60'

## 3 Class GRU Classification

In [31]:

        
        

nn_event_score2 = {}

for LAYERS in [1]: # , 2
    for L2 in [1e-5]:  # , 1e-4, 1e-6
        results = {'LAYERS': str(LAYERS), 'L2': L2}
        for i in range(10):
            for data_name, all_events_data in [
                        #                 ('reply', all_events_data_reply),
                                            ('all', all_events_data_all_3_class),
#                                                ('quote', all_events_data_quote)
                                              ]:
#                                                ('reply', all_events_data_reply)]:
                one_type_result = {}
                for event_name, all_event_data in all_events_data.items():
                    if True: #event_name == 'Student_Marches': #True #Student_Marches
                        print('i ', str(i), 'Event_name', event_name, 'LAYERS: ', str(LAYERS), 'L2: ', L2, len(all_event_data['train_ind']), len(all_event_data['test_ind']))

                        train_replies = []
                        train_sources = []
                        train_labels = []
                        for train_index in all_event_data['train_ind']:
                            train_replies.append(all_event_data['text'][train_index]['reply'])
                            train_sources.append(all_event_data['text'][train_index]['src'])
                            train_labels.append(all_event_data['info'][train_index]['label'])

                        test_replies = []
                        test_sources = []
                        test_labels = []
                        for test_index in all_event_data['test_ind']:
                            test_replies.append(all_event_data['text'][test_index]['reply'])
                            test_sources.append(all_event_data['text'][test_index]['src'])
                            test_labels.append(all_event_data['info'][test_index]['label'])


                        f1_micro, f1_macro,  f1_weighted, y_truths, y_classes = train_and_test( train_replies, train_sources, train_labels,
                                   test_replies, test_sources, test_labels,LABELS = {0:0, 1:1, 2: 2}, MODEL = 'GRU')

        #                 f1_macro = train_and_test(train_sources, train_replies, train_labels,
        #                            test_sources, test_replies, test_labels)


                        one_type_result[event_name] = {'f1_micro': f1_micro,
                                                       'f1_macro': f1_macro,
                                                       'f1_weighted': f1_weighted,
                                                       'y_truths': ' '.join([str(i) for i in y_truths]), 
                                                       'y_classes': ' '.join([str(i) for i in y_classes])}

                    if event_name not in nn_event_score2:
                        nn_event_score2[event_name] = [f1_macro]
                    else:
                        nn_event_score2[event_name].append(f1_macro)


                    results[data_name + '_' + str(i)] = one_type_result  

        results_str = json.dumps(results)
        write_result_to_file(results_str, './results/Replies_stance_classifier_using_GRU_3_class.csv')        
   



"{:.2f}&{:.2f}&{:.2f}&{:.2f}".format(np.mean(nn_event_score2['Student_Marches']),
                                     np.mean(nn_event_score2['Santa_Fe_Shooting']),
                                     np.mean(nn_event_score2['Iran_Deal']),
                             np.mean([nn_event_score2['Student_Marches'], nn_event_score2['Santa_Fe_Shooting'], nn_event_score2['Iran_Deal']]))



i  0 Event_name Santa_Fe_Shooting LAYERS:  1 L2:  1e-05 427 335
57
59
57
49
57
59
57
59
RNN / Embed / Sent = <class 'keras.layers.rnn.gru.GRU'>, 200, 200
GloVe / Trainable Word Embeddings = True, False
Build model...
Vocab size = 3632
Computing GloVe
Loading GloVe
Total number of null word embeddings:
676
Training
Epoch 1/20
6/6 [==============================] - 4s 208ms/step - loss: 1.4391 - accuracy: 0.4141 - f1_m: 0.3960 - val_loss: 1.0013 - val_accuracy: 0.7143 - val_f1_m: 0.0000e+00
Epoch 2/20
6/6 [==============================] - 1s 93ms/step - loss: 1.1354 - accuracy: 0.5130 - f1_m: 0.5088 - val_loss: 1.3492 - val_accuracy: 0.0000e+00 - val_f1_m: 0.0000e+00
Epoch 3/20
6/6 [==============================] - 1s 95ms/step - loss: 0.8899 - accuracy: 0.6328 - f1_m: 0.6128 - val_loss: 1.3392 - val_accuracy: 0.0000e+00 - val_f1_m: 0.0000e+00
Epoch 4/20
6/6 [==============================] - 1s 96ms/step - loss: 0.7329 - accuracy: 0.6953 - f1_m: 0.6881 - val_loss: 1.3602 - val_accurac

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


58
56
54
52
58
56
58
56
RNN / Embed / Sent = <class 'keras.layers.rnn.gru.GRU'>, 200, 200
GloVe / Trainable Word Embeddings = True, False
Build model...
Vocab size = 4247
Computing GloVe
Loading GloVe
Total number of null word embeddings:
890
Training
Epoch 1/20
8/8 [==============================] - 5s 239ms/step - loss: 1.6446 - accuracy: 0.3437 - f1_m: 0.2751 - val_loss: 1.1693 - val_accuracy: 0.0600 - val_f1_m: 0.0000e+00
Epoch 2/20
8/8 [==============================] - 1s 100ms/step - loss: 1.1267 - accuracy: 0.4945 - f1_m: 0.5124 - val_loss: 0.9765 - val_accuracy: 0.7000 - val_f1_m: 0.0000e+00
Epoch 3/20
8/8 [==============================] - 1s 103ms/step - loss: 1.0312 - accuracy: 0.5410 - f1_m: 0.4650 - val_loss: 0.9753 - val_accuracy: 0.7800 - val_f1_m: 0.0000e+00
Epoch 4/20
8/8 [==============================] - 1s 93ms/step - loss: 0.9605 - accuracy: 0.5654 - f1_m: 0.5246 - val_loss: 1.0577 - val_accuracy: 0.1000 - val_f1_m: 0.0000e+00
Epoch 5/20
8/8 [=====================

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


57
59
57
49
57
59
57
59
RNN / Embed / Sent = <class 'keras.layers.rnn.gru.GRU'>, 200, 200
GloVe / Trainable Word Embeddings = True, False
Build model...
Vocab size = 3632
Computing GloVe
Loading GloVe
Total number of null word embeddings:
676
Training
Epoch 1/20
6/6 [==============================] - 3s 215ms/step - loss: 1.5075 - accuracy: 0.4193 - f1_m: 0.3872 - val_loss: 1.1966 - val_accuracy: 0.0952 - val_f1_m: 0.0000e+00
Epoch 2/20
6/6 [==============================] - 1s 98ms/step - loss: 1.0952 - accuracy: 0.5573 - f1_m: 0.5279 - val_loss: 1.1315 - val_accuracy: 0.3571 - val_f1_m: 0.0000e+00
Epoch 3/20
6/6 [==============================] - 1s 93ms/step - loss: 0.8700 - accuracy: 0.6484 - f1_m: 0.6299 - val_loss: 1.2121 - val_accuracy: 0.0952 - val_f1_m: 0.0000e+00
Epoch 4/20
6/6 [==============================] - 1s 98ms/step - loss: 0.7319 - accuracy: 0.6927 - f1_m: 0.6922 - val_loss: 1.1608 - val_accuracy: 0.1667 - val_f1_m: 0.0000e+00
Epoch 5/20
6/6 [=======================

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


 6 Event_name Student_Marches LAYERS:  1 L2:  1e-05 283 210
55
53
50
47
55
53
55
53
RNN / Embed / Sent = <class 'keras.layers.rnn.gru.GRU'>, 200, 200
GloVe / Trainable Word Embeddings = True, False
Build model...
Vocab size = 2865
Computing GloVe
Loading GloVe
Total number of null word embeddings:
543
Training
Epoch 1/20
4/4 [==============================] - 4s 303ms/step - loss: 1.5347 - accuracy: 0.4055 - f1_m: 0.3768 - val_loss: 0.9278 - val_accuracy: 0.7500 - val_f1_m: 0.0690
Epoch 2/20
4/4 [==============================] - 0s 106ms/step - loss: 0.9926 - accuracy: 0.5866 - f1_m: 0.5618 - val_loss: 1.0652 - val_accuracy: 0.4643 - val_f1_m: 0.0000e+00
Epoch 3/20
4/4 [==============================] - 0s 108ms/step - loss: 0.9535 - accuracy: 0.6102 - f1_m: 0.6055 - val_loss: 1.0529 - val_accuracy: 0.4286 - val_f1_m: 0.0000e+00
Epoch 4/20
4/4 [==============================] - 0s 114ms/step - loss: 0.8080 - accuracy: 0.6890 - f1_m: 0.6760 - val_loss: 0.9745 - val_accuracy: 0.6429 - v

'0.53&0.54&0.47&0.52'